# Layer Visualizations
This notebook contains code to parse a saved TensorFlow model file and generate a pandas dataframe that contains information about all the layers.

In [1]:
import pandas as pd
import numpy as np
import re
from matplotlib import pyplot as plt
import plotly.express as px
from pathlib import Path

# enable plotly in VS Studio Code
import plotly.io as pio
#pio.renderers.default = "notebook_connected"
pio.renderers.default = "plotly_mimetype+notebook"

import wandb
from wandb.keras import WandbMetricsLogger, WandbModelCheckpoint

from workbench.utils.utils import create_filepaths, parse_model_name, append_dict_to_csv
from workbench.config.config import initialize


In [2]:
# Configure pandas to show all columns & rows
pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)

In [3]:
model_name = "shufflenetv1_1_96_c3_o3_000"
#model_name = "mobilenetv2_0.1_96_c3_o3_keras"
#model_name = "efficientNetB0_0.1_96_c3_o3_keras"

In [4]:
model_dict = {}

In [5]:
parse_model_name(model_name)

('shufflenetv1', '1', '96', 'c3', 'o3', '000')

In [6]:
models_dir = initialize()

In [7]:
models_path, models_summary_path, models_image_path, models_layer_df_path, models_tf_path, models_tflite_path, models_tflite_opt_path = create_filepaths(model_name)

i:\tinyml\tiny_cnn\models


In [8]:
filepath = models_dir.joinpath(f"{model_name}/{model_name}_layers.pkl")

#filepath = f"i:/tinyml/tiny_cnn/models/{model_name}/{model_name}_layers.pkl"
df = pd.read_pickle(filepath)

In [9]:
df_filepath = models_dir.joinpath(f"{model_name}/{model_name}.pkl")

In [10]:
df.columns

Index(['index', 'name', 'layer_type', 'input_shape', 'input', 'input_node_1',
       'output_shape', 'dtype', 'params', 'variables', 'weights_shape',
       'weights_name', 'weights_dtype', '#_activations', 'OPS', 'MACS',
       'filters', 'trainable', 'kernel_size', 'strides', 'padding',
       'dilation_rate', 'groups', 'activation', 'use_bias', 'data_format',
       'input_node_type_1', 'input_node_2', 'input_node_type_2'],
      dtype='object')

In [11]:
df.head(10)

,index,name,layer_type,input_shape,input,input_node_1,output_shape,dtype,params,variables,weights_shape,weights_name,weights_dtype,#_activations,OPS,MACS,filters,trainable,kernel_size,strides,padding,dilation_rate,groups,activation,use_bias,data_format,input_node_type_1,input_node_2,input_node_type_2
0,0,input_2,<class 'keras.engine.input_layer.InputLayer'>,"[(None, 96, 96, 3)]","KerasTensor(type_spec=TensorSpec(shape=(None, ...",input_2,"[(None, 96, 96, 3)]",float32,0,0,0,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN
1,1,conv2d_33,<class 'keras.layers.convolutional.conv2d.Conv...,"(None, 96, 96, 3)","KerasTensor(type_spec=TensorSpec(shape=(None, ...",input_2,"(None, 48, 48, 24)",float32,672,2,"(3, 3, 3, 24)",conv2d_33/kernel:0,<dtype: 'float32'>,NaN,3041280,1492992,24,True,"(3, 3)","(2, 2)",same,"(1, 1)",1,linear,True,channels_last,NaN,NaN,NaN
2,2,batch_normalization_49,<class 'keras.layers.normalization.batch_norma...,"(None, 48, 48, 24)","KerasTensor(type_spec=TensorSpec(shape=(None, ...",conv2d_33/BiasAdd:0,"(None, 48, 48, 24)",float32,96,4,(24),batch_normalization_49/gamma:0,<dtype: 'float32'>,NaN,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN
3,3,re_lu_33,<class 'keras.layers.activation.relu.ReLU'>,"(None, 48, 48, 24)","KerasTensor(type_spec=TensorSpec(shape=(None, ...",batch_normalization_49/FusedBatchNormV3:0,"(None, 48, 48, 24)",float32,0,0,0,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN
4,4,max_pooling2d_1,<class 'keras.layers.pooling.max_pooling2d.Max...,"(None, 48, 48, 24)","KerasTensor(type_spec=TensorSpec(shape=(None, ...",re_lu_33/Relu:0,"(None, 24, 24, 24)",float32,0,0,0,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN
5,5,conv2d_34,<class 'keras.layers.convolutional.conv2d.Conv...,"(None, 24, 24, 24)","KerasTensor(type_spec=TensorSpec(shape=(None, ...",max_pooling2d_1/MaxPool:0,"(None, 24, 24, 50)",float32,650,2,"(1, 1, 12, 50)",conv2d_34/kernel:0,<dtype: 'float32'>,NaN,1411200,691200,50,True,"(1, 1)","(1, 1)",same,"(1, 1)",2,linear,True,channels_last,NaN,NaN,NaN
6,6,batch_normalization_50,<class 'keras.layers.normalization.batch_norma...,"(None, 24, 24, 50)","KerasTensor(type_spec=TensorSpec(shape=(None, ...",conv2d_34/BiasAdd:0,"(None, 24, 24, 50)",float32,200,4,(50),batch_normalization_50/gamma:0,<dtype: 'float32'>,NaN,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN
7,7,re_lu_34,<class 'keras.layers.activation.relu.ReLU'>,"(None, 24, 24, 50)","KerasTensor(type_spec=TensorSpec(shape=(None, ...",batch_normalization_50/FusedBatchNormV3:0,"(None, 24, 24, 50)",float32,0,0,0,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN
8,8,reshape_32,<class 'keras.layers.reshaping.reshape.Reshape'>,"(None, 24, 24, 50)","KerasTensor(type_spec=TensorSpec(shape=(None, ...",re_lu_34/Relu:0,"(None, 24, 24, 25, 2)",float32,0,0,0,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN
9,9,permute_16,<class 'keras.layers.reshaping.permute.Permute'>,"(None, 24, 24, 25, 2)","KerasTensor(type_spec=TensorSpec(shape=(None, ...",reshape_32/Reshape:0,"(None, 24, 24, 2, 25)",float32,0,0,0,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN


## Helper functions

In [12]:
def split_tuples(x):
    x = str(x).strip("()")
    x = x.split(",")
    return x

In [13]:
def return_batch_size(x):
    x = str(x).strip("()")
    split_strings = x.split(",")
    #print(split_strings)
    return split_strings[0].strip()

In [14]:
def return_input_height(x):
    #image_size=(img_height, img_width),

    x = str(x).strip("()")
    split_strings = x.split(",")
    if len(split_strings) in [3,4]:
        return split_strings[1].strip()
    else:
        return "None"


In [15]:
def return_input_width(x):
    #image_size=(img_height, img_width),

    x = str(x).strip("()")
    split_strings = x.split(",")
    if len(split_strings) in [4]:
        return split_strings[2].strip()
    else:
        return "None"


In [16]:
def return_channels(x):
    x = str(x).strip("()")
    split_strings = x.split(",")
    #print(split_strings)

    return split_strings[-1].strip()

In [17]:
def split_shape_columns(df, colum_name):
    df[colum_name] = df[colum_name].map(str)
    df[colum_name] = df[colum_name].str.strip("[]").replace('\)\,', ')*', regex=True)
    df_cols = df[colum_name].str.split("*", expand=True)
    prefix = colum_name.split("_")[0]
    df_cols.columns = [f"{prefix}_{x+1}" for x in df_cols.columns]
    return df_cols

In [18]:
# df["input_shape"].unique()

In [19]:
# split_df = df["input_shape"].map(str)
# split_df.unique

# Input shapes & activations

In [20]:
df_input = split_shape_columns(df, "input_shape")
no_input_cols = len(df_input.columns)
for input in range(1,no_input_cols+1):
    df_input[f"b_i_{input}"]= df_input[f"input_{input}"].apply(return_batch_size)
    df_input[f"h_i_{input}"] = df_input[f"input_{input}"].apply(return_input_height)
    df_input[f"w_i_{input}"] = df_input[f"input_{input}"].apply(return_input_width)
    df_input[f"c_i_{input}"] = df_input[f"input_{input}"].apply(return_channels)


input_cols = df_input.columns.to_list()

#df_input["w_i_2"].replace('None', np.nan, inplace=True)
df_input["b_i_1"].replace('None', np.nan, inplace=True)
df_input["w_i_1"].replace('None', np.nan, inplace=True)
df_input["h_i_1"].replace('None', np.nan, inplace=True)
df_input["c_i_1"].replace('None', np.nan, inplace=True)

if "b_i_2" in input_cols:
    df_input["b_i_2"] = df_input["b_i_2"].str.replace("\(", "")
    df_input["b_i_2"].replace('None', np.nan, inplace=True)
    df_input["w_i_2"].replace('None', np.nan, inplace=True)
    df_input["h_i_2"].replace('None', np.nan, inplace=True)
    df_input["c_i_2"].replace('None', np.nan, inplace=True)
df_input.fillna(1, inplace=True)

# cast to int
df_input["w_i_1"] = df_input["w_i_1"].map(int)
df_input["h_i_1"] = df_input["h_i_1"].map(int)
df_input["b_i_1"] = df_input["b_i_1"].map(int)
df_input["c_i_1"] = df_input["c_i_1"].map(int)

if "b_i_2" in input_cols:
    df_input["b_i_2"] = df_input["b_i_2"].map(int)
    df_input["w_i_2"] = df_input["w_i_2"].map(int)
    df_input["h_i_2"] = df_input["h_i_2"].map(int)
    df_input["c_i_2"] = df_input["c_i_2"].map(int)

C:\Users\Susanne\AppData\Local\Temp\ipykernel_10012\1746192384.py:19: FutureWarning:

The default value of regex will change from True to False in a future version.



In [21]:
# df_input = split_shape_columns(df, "input_shape")
# df_input

In [22]:
# #df["input_shape"][0] =(None, None, None, None)
# df["input_shape"] = df["input_shape"].map(str)
# df["input_shape"] = df["input_shape"].str.strip("[]").replace('\)\,', ')*', regex=True)
# df_input = df["input_shape"].str.split("*", expand=True)
# df_input.columns =[f"input_{x+1}" for x in df_input.columns]
# no_input_cols = len(df_input.columns)
# print(no_input_cols)
# df_input


In [23]:
#df_input["input_1_split"] = df_input["input_1"].apply(split_tuples)

In [24]:
# no_input_cols = len(df_input.columns)

In [25]:
# for input in range(1,no_input_cols+1):
#     df_input[f"b_i_{input}"]= df_input[f"input_{input}"].apply(return_batch_size)
#     df_input[f"h_i_{input}"] = df_input[f"input_{input}"].apply(return_input_height)
#     df_input[f"w_i_{input}"] = df_input[f"input_{input}"].apply(return_input_width)
#     df_input[f"c_i_{input}"] = df_input[f"input_{input}"].apply(return_channels)

# df_input

In [26]:
# input_cols = df_input.columns.to_list()


In [27]:
# #df_input["w_i_2"].replace('None', np.nan, inplace=True)
# df_input["b_i_1"].replace('None', np.nan, inplace=True)
# df_input["w_i_1"].replace('None', np.nan, inplace=True)
# df_input["h_i_1"].replace('None', np.nan, inplace=True)
# df_input["c_i_1"].replace('None', np.nan, inplace=True)

# if "b_i_2" in input_cols:
#     df_input["b_i_2"] = df_input["b_i_2"].str.replace("\(", "")
#     df_input["b_i_2"].replace('None', np.nan, inplace=True)
#     df_input["w_i_2"].replace('None', np.nan, inplace=True)
#     df_input["h_i_2"].replace('None', np.nan, inplace=True)
#     df_input["c_i_2"].replace('None', np.nan, inplace=True)
# df_input.fillna(1, inplace=True)


In [28]:
# df_input

In [29]:
#df_input.c_i_1.value_counts()

In [30]:

# df_input["w_i_1"] = df_input["w_i_1"].map(int)
# df_input["h_i_1"] = df_input["h_i_1"].map(int)
# df_input["b_i_1"] = df_input["b_i_1"].map(int)
# df_input["c_i_1"] = df_input["c_i_1"].map(int)

# if "b_i_2" in input_cols:
#     df_input["b_i_2"] = df_input["b_i_2"].map(int)
#     df_input["w_i_2"] = df_input["w_i_2"].map(int)
#     df_input["h_i_2"] = df_input["h_i_2"].map(int)
#     df_input["c_i_2"] = df_input["c_i_2"].map(int)

In [31]:

# df_input["b_i_1"] = df_input["b_i_1"].map(int)
# df_input["w_i_1"] = df_input["w_i_1"].map(int)
# df_input["h_i_1"] = df_input["h_i_1"].map(int)
# df_input["c_i_1"] = df_input["c_i_1"].map(int)

# df_input["b_i_2"] = df_input["b_i_2"].map(int)
# df_input["w_i_2"] = df_input["w_i_2"].map(int)
# df_input["h_i_2"] = df_input["h_i_2"].map(int)
# df_input["c_i_2"] = df_input["c_i_2"].map(int)

In [32]:
#df_input

# Output shape and activations

In [33]:
df_output =split_shape_columns(df, "output_shape") # TODO: This split does not work for EfficientNet
no_output_cols = len(df_output.columns)
# split output dimensions
for output in range(1,no_output_cols+1):
    df_output[f"b_o_{output}"]= df_output[f"output_{output}"].apply(return_batch_size)
    df_output[f"h_o_{output}"] = df_output[f"output_{output}"].apply(return_input_height)
    df_output[f"w_o_{output}"] = df_output[f"output_{output}"].apply(return_input_width)
    df_output[f"c_o_{output}"] = df_output[f"output_{output}"].apply(return_channels)

# fill NaN with 1 for mulitplication
df_output["b_o_1"].replace('None', np.nan, inplace=True)
df_output["w_o_1"].replace('None', np.nan, inplace=True)
df_output["h_o_1"].replace('None', np.nan, inplace=True)
df_output["c_o_1"].replace('None', np.nan, inplace=True)
df_output.fillna(1, inplace=True)

df_output["w_o_1"] = df_output["w_o_1"].map(int)
df_output["h_o_1"] = df_output["h_o_1"].map(int)
df_output["b_o_1"] = df_output["b_o_1"].map(int)
df_output["c_o_1"] = df_output["c_o_1"].map(int)

In [34]:
# df_output = split_shape_columns(df, "output_shape")
# df_output

In [35]:
# no_output_cols = len(df_output.columns)
# no_output_cols

In [36]:
# for output in range(1,no_output_cols+1):
#     df_output[f"b_o_{output}"]= df_output[f"output_{output}"].apply(return_batch_size)
#     df_output[f"h_o_{output}"] = df_output[f"output_{output}"].apply(return_input_height)
#     df_output[f"w_o_{output}"] = df_output[f"output_{output}"].apply(return_input_width)
#     df_output[f"c_o_{output}"] = df_output[f"output_{output}"].apply(return_channels)

# df_output

In [37]:
# df_output["b_o_1"].replace('None', np.nan, inplace=True)
# df_output["w_o_1"].replace('None', np.nan, inplace=True)
# df_output["h_o_1"].replace('None', np.nan, inplace=True)
# df_output["c_o_1"].replace('None', np.nan, inplace=True)
# df_output.fillna(1, inplace=True)

In [38]:
# df_output["w_o_1"] = df_output["w_o_1"].map(int)
# df_output["h_o_1"] = df_output["h_o_1"].map(int)
# df_output["b_o_1"] = df_output["b_o_1"].map(int)
# df_output["c_o_1"] = df_output["c_o_1"].map(int)

# Feature Engineering

In [39]:
def get_unique_values_without_nan(df,col_name):
    """Retrieve list of unique column values, excluding nan

    Args:
        df (pd.DataFrame): pandas dataframe that contains the required column
        col_name (string): name of a column in the pandas dataframe

    Returns:
        list: list of extracted unique values of the specified column
    """
    try:
        value_list = df[col_name].unique().tolist()
        # exclude nan
        value_list = [item for item in value_list if str(item) != 'nan']
        return value_list
    except:
        print(f"WARNING: Colum name {col_name} is not in this dataframe!")

        return []

In [40]:
df = df.join(df_input)

In [41]:
df = df.join(df_output)

In [42]:
name_code ="i_2"

In [43]:
def calculate_activations(x, name_code):
    h_i = int(x[f"h_{name_code}"])
    w_i = int(x[f"w_{name_code}"])
    c_i = int(x[f"c_{name_code}"])
    out = h_i * w_i * c_i
    if out >1:
        return out
    else:
        return 0

In [44]:
df["#i_1_activations"] = df.apply(calculate_activations, name_code= "i_1", axis=1)#df_input["h_i_1"] * df_input["w_i_1"] * df_input["c_i_1"]
df["#o_1_activations"] = df.apply(calculate_activations, name_code= "o_1", axis=1)#  df_output["h_o_1"] * df_output["w_o_1"] * df_output["c_o_1"]

In [45]:
if "b_i_2" in input_cols:
    df["#i_2_activations"] = df.apply(calculate_activations, name_code="i_2", axis=1)

In [46]:
df["#i_2_activations"].value_counts()

0        186
14400      7
7200       4
28800      3
3456       1
3600       1
Name: #i_2_activations, dtype: int64

In [47]:
if "#i_2_activations" in df.columns:
    df["peak_activations"] = df["#i_1_activations"] + df["#i_2_activations"] + df["#o_1_activations"]
else:
    df["peak_activations"] = df["#i_1_activations"]  + df["#o_1_activations"]    

In [48]:
unique_values = get_unique_values_without_nan(df, "input_node_2")
unique_values

['average_pooling2d_3',
 're_lu_35',
 're_lu_37',
 're_lu_39',
 'average_pooling2d_4',
 're_lu_43',
 're_lu_45',
 're_lu_47',
 're_lu_49',
 're_lu_51',
 're_lu_53',
 're_lu_55',
 'average_pooling2d_5',
 're_lu_59',
 're_lu_61',
 're_lu_63']

In [49]:
# fill activations from start node to end node

for item in unique_values:
    first_idx = df.loc[df["name"] == item].index[0]
    last_idx = df.loc[df["input_node_2"] == item].index[0]

    activations = df["#i_2_activations"][last_idx]

    # fill the corresponding rows with the extracted information
    df.loc[first_idx: last_idx, "#i_2_activations"] = activations
print(df[first_idx: last_idx+1])

     index                    name  \
187    187                re_lu_63   
188    188               conv2d_64   
189    189  batch_normalization_95   
190    190                re_lu_64   
191    191              reshape_62   
192    192              permute_31   
193    193              reshape_63   
194    194     depthwise_conv2d_31   
195    195  batch_normalization_96   
196    196               conv2d_65   
197    197  batch_normalization_97   
198    198                  add_25   

                                            layer_type  \
187        <class 'keras.layers.activation.relu.ReLU'>   
188  <class 'keras.layers.convolutional.conv2d.Conv...   
189  <class 'keras.layers.normalization.batch_norma...   
190        <class 'keras.layers.activation.relu.ReLU'>   
191   <class 'keras.layers.reshaping.reshape.Reshape'>   
192   <class 'keras.layers.reshaping.permute.Permute'>   
193   <class 'keras.layers.reshaping.reshape.Reshape'>   
194  <class 'keras.layers.convolutional.d

In [50]:
df.columns

Index(['index', 'name', 'layer_type', 'input_shape', 'input', 'input_node_1',
       'output_shape', 'dtype', 'params', 'variables', 'weights_shape',
       'weights_name', 'weights_dtype', '#_activations', 'OPS', 'MACS',
       'filters', 'trainable', 'kernel_size', 'strides', 'padding',
       'dilation_rate', 'groups', 'activation', 'use_bias', 'data_format',
       'input_node_type_1', 'input_node_2', 'input_node_type_2', 'input_1',
       'input_2', 'b_i_1', 'h_i_1', 'w_i_1', 'c_i_1', 'b_i_2', 'h_i_2',
       'w_i_2', 'c_i_2', 'output_1', 'b_o_1', 'h_o_1', 'w_o_1', 'c_o_1',
       '#i_1_activations', '#o_1_activations', '#i_2_activations',
       'peak_activations'],
      dtype='object')

In [51]:
df["total_params"] = df["params"].cumsum()
df["total_MACs"] = df["MACS"].cumsum()
df["total_OPS"] = df["OPS"].cumsum()

In [52]:
# df["total_params"].max()
# df["peak_activations"].argmax()

In [53]:
model_dict["model_name"] = model_name
model_dict["total_params"] = df["total_params"].max()
model_dict["total_MACs"] = df["total_MACs"].max()
model_dict["total OPS"] = df["total_OPS"].max()
model_dict["peak_activations"] = df["peak_activations"].max()
model_dict["index_peak_activations"] = df["peak_activations"].argmax()

In [54]:
model_dict

{'model_name': 'shufflenetv1_1_96_c3_o3_000',
 'total_params': 965251,
 'total_MACs': 48674592,
 'total OPS': 97816824,
 'peak_activations': 110592,
 'index_peak_activations': 2}

In [55]:
csv_file_name = "model_metrics.csv"
csv_path = Path.cwd().joinpath(csv_file_name)
append_dict_to_csv(csv_path, model_dict)

In [56]:
df.to_pickle(df_filepath)

In [57]:
df.columns

Index(['index', 'name', 'layer_type', 'input_shape', 'input', 'input_node_1',
       'output_shape', 'dtype', 'params', 'variables', 'weights_shape',
       'weights_name', 'weights_dtype', '#_activations', 'OPS', 'MACS',
       'filters', 'trainable', 'kernel_size', 'strides', 'padding',
       'dilation_rate', 'groups', 'activation', 'use_bias', 'data_format',
       'input_node_type_1', 'input_node_2', 'input_node_type_2', 'input_1',
       'input_2', 'b_i_1', 'h_i_1', 'w_i_1', 'c_i_1', 'b_i_2', 'h_i_2',
       'w_i_2', 'c_i_2', 'output_1', 'b_o_1', 'h_o_1', 'w_o_1', 'c_o_1',
       '#i_1_activations', '#o_1_activations', '#i_2_activations',
       'peak_activations', 'total_params', 'total_MACs', 'total_OPS'],
      dtype='object')

In [58]:
df.head(12)

,index,name,layer_type,input_shape,input,input_node_1,output_shape,dtype,params,variables,weights_shape,weights_name,weights_dtype,#_activations,OPS,MACS,filters,trainable,kernel_size,strides,padding,dilation_rate,groups,activation,use_bias,data_format,input_node_type_1,input_node_2,input_node_type_2,input_1,input_2,b_i_1,h_i_1,w_i_1,c_i_1,b_i_2,h_i_2,w_i_2,c_i_2,output_1,b_o_1,h_o_1,w_o_1,c_o_1,#i_1_activations,#o_1_activations,#i_2_activations,peak_activations,total_params,total_MACs,total_OPS
0,0,input_2,<class 'keras.engine.input_layer.InputLayer'>,"(None, 96, 96, 3)","KerasTensor(type_spec=TensorSpec(shape=(None, ...",input_2,"(None, 96, 96, 3)",float32,0,0,0,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,"(None, 96, 96, 3)",1,1,96,96,3,1,1,1,1,"(None, 96, 96, 3)",1,96,96,3,27648,27648,0,55296,0,0,0
1,1,conv2d_33,<class 'keras.layers.convolutional.conv2d.Conv...,"(None, 96, 96, 3)","KerasTensor(type_spec=TensorSpec(shape=(None, ...",input_2,"(None, 48, 48, 24)",float32,672,2,"(3, 3, 3, 24)",conv2d_33/kernel:0,<dtype: 'float32'>,NaN,3041280,1492992,24,True,"(3, 3)","(2, 2)",same,"(1, 1)",1,linear,True,channels_last,NaN,NaN,NaN,"(None, 96, 96, 3)",1,1,96,96,3,1,1,1,1,"(None, 48, 48, 24)",1,48,48,24,27648,55296,0,82944,672,1492992,3041280
2,2,batch_normalization_49,<class 'keras.layers.normalization.batch_norma...,"(None, 48, 48, 24)","KerasTensor(type_spec=TensorSpec(shape=(None, ...",conv2d_33/BiasAdd:0,"(None, 48, 48, 24)",float32,96,4,(24),batch_normalization_49/gamma:0,<dtype: 'float32'>,NaN,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,"(None, 48, 48, 24)",1,1,48,48,24,1,1,1,1,"(None, 48, 48, 24)",1,48,48,24,55296,55296,0,110592,768,1492992,3041280
3,3,re_lu_33,<class 'keras.layers.activation.relu.ReLU'>,"(None, 48, 48, 24)","KerasTensor(type_spec=TensorSpec(shape=(None, ...",batch_normalization_49/FusedBatchNormV3:0,"(None, 48, 48, 24)",float32,0,0,0,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,"(None, 48, 48, 24)",1,1,48,48,24,1,1,1,1,"(None, 48, 48, 24)",1,48,48,24,55296,55296,0,110592,768,1492992,3041280
4,4,max_pooling2d_1,<class 'keras.layers.pooling.max_pooling2d.Max...,"(None, 48, 48, 24)","KerasTensor(type_spec=TensorSpec(shape=(None, ...",re_lu_33/Relu:0,"(None, 24, 24, 24)",float32,0,0,0,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,"(None, 48, 48, 24)",1,1,48,48,24,1,1,1,1,"(None, 24, 24, 24)",1,24,24,24,55296,13824,0,69120,768,1492992,3041280
5,5,conv2d_34,<class 'keras.layers.convolutional.conv2d.Conv...,"(None, 24, 24, 24)","KerasTensor(type_spec=TensorSpec(shape=(None, ...",max_pooling2d_1/MaxPool:0,"(None, 24, 24, 50)",float32,650,2,"(1, 1, 12, 50)",conv2d_34/kernel:0,<dtype: 'float32'>,NaN,1411200,691200,50,True,"(1, 1)","(1, 1)",same,"(1, 1)",2,linear,True,channels_last,NaN,NaN,NaN,"(None, 24, 24, 24)",1,1,24,24,24,1,1,1,1,"(None, 24, 24, 50)",1,24,24,50,13824,28800,0,42624,1418,2184192,4452480
6,6,batch_normalization_50,<class 'keras.layers.normalization.batch_norma...,"(None, 24, 24, 50)","KerasTensor(type_spec=TensorSpec(shape=(None, ...",conv2d_34/BiasAdd:0,"(None, 24, 24, 50)",float32,200,4,(50),batch_normalization_50/gamma:0,<dtype: 'float32'>,NaN,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,"(None, 24, 24, 50)",1,1,24,24,50,1,1,1,1,"(None, 24, 24, 50)",1,24,24,50,28800,28800,0,57600,1618,2184192,4452480
7,7,re_lu_34,<class 'keras.layers.activation.relu.ReLU'>,"(None, 24, 24, 50)","KerasTensor(type_spec=TensorSpec(shape=(None, ...",batch_normalization_50/FusedBatchNormV3:0,"(None, 24, 24, 50)",float32,0,0,0,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,"(None, 24, 24, 50)",1,1,24,24,50,1,1,1,1,"(None, 24, 24, 50)",1,24,24,50,28800,28800,0,57600,1618,2184192,4452480
8,8,reshape_32,<class 'keras.layers.reshaping.reshape.Reshape'>,"(None, 24, 24, 50)","KerasTensor(type_spec=TensorSpec(shape=(None, ...",re_lu_34/Relu:0,"(None, 24, 24, 25, 2)",float32,0,0,0,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,"(None, 24, 24, 50)",1,1,24,24,50,1,1,1,1,"(None, 24, 24, 25, 2)",1,1,1,2,28800,2,0,28802,1618,2184192,4452480
9,9,permute_16,<

In [59]:
df["input_shape"].unique()

array(['(None, 96, 96, 3)', '(None, 48, 48, 24)', '(None, 24, 24, 24)',
       '(None, 24, 24, 50)', '(None, 24, 24, 25, 2)',
       '(None, 24, 24, 2, 25)', '(None, 12, 12, 50)',
       '(None, 12, 12, 176)', '(None, 12, 12, 176)* (None, 12, 12, 24)',
       '(None, 12, 12, 200)', '(None, 12, 12, 25, 2)',
       '(None, 12, 12, 2, 25)',
       '(None, 12, 12, 200)* (None, 12, 12, 200)', '(None, 12, 12, 100)',
       '(None, 12, 12, 50, 2)', '(None, 12, 12, 2, 50)',
       '(None, 6, 6, 100)', '(None, 6, 6, 200)',
       '(None, 6, 6, 200)* (None, 6, 6, 200)', '(None, 6, 6, 400)',
       '(None, 6, 6, 50, 2)', '(None, 6, 6, 2, 50)',
       '(None, 6, 6, 400)* (None, 6, 6, 400)', '(None, 6, 6, 100, 2)',
       '(None, 6, 6, 2, 100)', '(None, 3, 3, 200)', '(None, 3, 3, 400)',
       '(None, 3, 3, 400)* (None, 3, 3, 400)', '(None, 3, 3, 800)',
       '(None, 3, 3, 100, 2)', '(None, 3, 3, 2, 100)',
       '(None, 3, 3, 800)* (None, 3, 3, 800)', '(None, 800)'],
      dtype=object)

# Working with Activations

In [60]:
# unique_values = get_unique_values_without_nan(df, "input_node_0")
# unique_values

In [61]:

# # fill activations from start node to end node

# for item in unique_values:
#     first_idx = df.loc[df["name"] == item].index[0]
#     last_idx = df.loc[df["input_node_0"] == item].index[0]

#     activations = df["#i_2_activations"][last_idx]

#     # fill the corresponding rows with the extracted information
#     df.loc[first_idx: last_idx, "#i_2_activations"] = activations
#     #print(df[first_idx: last_idx+1])

In [62]:
# df.columns

In [63]:
# activation_cols = ["index", "name", "input_shape", "input_node_1", "#i_1_activations" , "input_node_0", "#i_2_activations", "input_shape", "input_node_type_1", "input_node_type_0"]
# df_activations = df[activation_cols].copy()
# df_activations.head(20)

In [64]:
# # fill activations from start node to end node

# for item in second_inputs:
#     first_idx = df_activations.loc[df_activations["name"] == item].index[0]
#     last_idx = df_activations.loc[df_activations["input_node_0"] == item].index[0]

#     activations = df_activations["#i_2_activations"][last_idx]
#     df_activations.loc[first_idx: last_idx, "#i_2_activations"] = activations
#     #print(df_activations[first_idx: last_idx+1])
    

    

In [65]:
#input_name = "block_4_project_BN"

In [67]:
# first_idx = df.loc[df["name"] == input_name].index[0]
# last_idx = df.loc[df["input_node_2"] == input_name].index[0]
# last_idx

In [68]:
df[first_idx: last_idx+1]

,index,name,layer_type,input_shape,input,input_node_1,output_shape,dtype,params,variables,weights_shape,weights_name,weights_dtype,#_activations,OPS,MACS,filters,trainable,kernel_size,strides,padding,dilation_rate,groups,activation,use_bias,data_format,input_node_type_1,input_node_2,input_node_type_2,input_1,input_2,b_i_1,h_i_1,w_i_1,c_i_1,b_i_2,h_i_2,w_i_2,c_i_2,output_1,b_o_1,h_o_1,w_o_1,c_o_1,#i_1_activations,#o_1_activations,#i_2_activations,peak_activations,total_params,total_MACs,total_OPS
187,187,re_lu_63,<class 'keras.layers.activation.relu.ReLU'>,"(None, 3, 3, 800)","KerasTensor(type_spec=TensorSpec(shape=(None, ...",add_24/add:0,"(None, 3, 3, 800)",float32,0,0,0,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,"(None, 3, 3, 800)",1,1,3,3,800,1,1,1,1,"(None, 3, 3, 800)",1,3,3,800,7200,7200,7200,14400,795048,45778392,92013624
188,188,conv2d_64,<class 'keras.layers.convolutional.conv2d.Conv...,"(None, 3, 3, 800)","KerasTensor(type_spec=TensorSpec(shape=(None, ...",re_lu_63/Relu:0,"(None, 3, 3, 200)",float32,80200,2,"(1, 1, 400, 200)",conv2d_64/kernel:0,<dtype: 'float32'>,NaN,2881800,1440000,200,True,"(1, 1)","(1, 1)",same,"(1, 1)",2,linear,True,channels_last,NaN,NaN,NaN,"(None, 3, 3, 800)",1,1,3,3,800,1,1,1,1,"(None, 3, 3, 200)",1,3,3,200,7200,1800,7200,9000,875248,47218392,94895424
189,189,batch_normalization_95,<class 'keras.layers.normalization.batch_norma...,"(None, 3, 3, 200)","KerasTensor(type_spec=TensorSpec(shape=(None, ...",conv2d_64/BiasAdd:0,"(None, 3, 3, 200)",float32,800,4,(200),batch_normalization_95/gamma:0,<dtype: 'float32'>,NaN,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,"(None, 3, 3, 200)",1,1,3,3,200,1,1,1,1,"(None, 3, 3, 200)",1,3,3,200,1800,1800,7200,3600,876048,47218392,94895424
190,190,re_lu_64,<class 'keras.layers.activation.relu.ReLU'>,"(None, 3, 3, 200)","KerasTensor(type_spec=TensorSpec(shape=(None, ...",batch_normalization_95/FusedBatchNormV3:0,"(None, 3, 3, 200)",float32,0,0,0,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,"(None, 3, 3, 200)",1,1,3,3,200,1,1,1,1,"(None, 3, 3, 200)",1,3,3,200,1800,1800,7200,3600,876048,47218392,94895424
191,191,reshape_62,<class 'keras.layers.reshaping.reshape.Reshape'>,"(None, 3, 3, 200)","KerasTensor(type_spec=TensorSpec(shape=(None, ...",re_lu_64/Relu:0,"(None, 3, 3, 100, 2)",float32,0,0,0,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,"(None, 3, 3, 200)",1,1,3,3,200,1,1,1,1,"(None, 3, 3, 100, 2)",1,1,1,2,1800,2,7200,1802,876048,47218392,94895424
192,192,permute_31,<class 'keras.layers.reshaping.permute.Permute'>,"(None, 3, 3, 100, 2)","KerasTensor(type_spec=TensorSpec(shape=(None, ...",reshape_62/Reshape:0,"(None, 3, 3, 2, 100)",float32,0,0,0,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,"(None, 3, 3, 100, 2)",1,1,1,1,2,1,1,1,1,"(None, 3, 3, 2, 100)",1,1,1,100,2,100,7200,102,876048,47218392,94895424
193,193,reshape_63,<class 'keras.layers.reshaping.reshape.Reshape'>,"(None, 3, 3, 2, 100)","KerasTensor(type_spec=TensorSpec(shape=(None, ...",permute_31/transpose:0,"(None, 3, 3, 200)",float32,0,0,0,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,"(None, 3, 3, 2, 100)",1,1,1,1,100,1,1,1,1,"(None, 3, 3, 200)",1,3,3,200,100,1800,7200,1900,876048,47218392,94895424
194,194,depthwise_conv2d_31,<class 'keras.layers.convolutional.depthwise_c...,"(None, 3, 3, 200)","KerasTensor(type_spec=TensorSpec(shape=(None, ...",reshape_63/Reshape:0,"(None, 3, 3, 200)",float32,2000,2,"(3, 3, 200, 1)",depthwise_conv2d_31/depthwise_kernel:0,<dtype: 'float32'>,NaN,34200,16200,0,True,"(3, 3)","(1, 1)",same,"(1, 1)",1,linear,True,channels_last,NaN,NaN,NaN,"(None, 3, 3, 200)",1,1,3,3,200,1,1,1,1,"(None, 3, 3, 200)",1,3,3,200,1800,1800,7200,3600,878048,47234592,94929624
195,195,batch_normalization_96,<class 'keras.layers.normalization.batch_norma...,"(None, 3, 3, 200)","KerasTensor(type_spec=TensorSpec(shape=(None, ...",depthwise_conv2d_31/BiasAdd:0,"(None, 3, 3, 200)",float32,800,4,(200),batch_normalization_96/gamma:0,<dtype: 'float32'>,NaN,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,"(None, 3, 3, 200)",1,1,3,3,200,1,

In [69]:
# df_activations.loc[last_idx]

In [70]:
# activations = df_activations["#i_2_activations"][last_idx]
# activations

In [71]:
# df_activations.loc[first_idx: last_idx, "#i_2_activations"] = activations
# df_activations.head(30)

# Save cleaned dataframe

In [72]:
clean_df_filepath = models_dir.joinpath(f"{model_name}/{model_name}_layers_clean.pkl")

#filepath = f"i:/tinyml/tiny_cnn/models/{model_name}/{model_name}_layers.pkl"
df.to_pickle(clean_df_filepath)

# Load tflite model data

In [11]:
# Get dataframes from benchmarking tflite model 
models_peak_memory_df_path = models_dir.joinpath(model_name, "peak_memory_df")
models_peak_memory_df_path

tensor_info_df_path = models_dir.joinpath(model_name, "tensor_info_df")
tensor_info_df_path

WindowsPath('i:/tinyml/tiny_cnn/models/shufflenetv1_1_96_c3_o3_000/tensor_info_df')

In [12]:
tensor_info_df = pd.read_pickle(tensor_info_df_path)
tensor_info_df

,tensor_id,tensor,shape,size_in_RAM_kb
1,0,serving_default_input_2:0,"(1, 96, 96, 3)",27.648
2,114,shufflenetv1/re_lu_33/Relu;shufflenetv.../Bias...,"(1, 48, 48, 24)",55.296
3,115,shufflenetv1/max_pooling2d_1/MaxPool,"(1, 24, 24, 24)",13.824
4,116,shufflenetv1/average_pooling2d_3/AvgPool,"(1, 12, 12, 24)",3.456
5,117,shufflenetv1/re_lu_34/Relu;shufflenetv...1/dep...,"(1, 24, 24, 50)",28.800
...,...,...,...,...
217,329,shufflenetv1/re_lu_65/Relu;shufflenetv1/add_25...,"(1, 3, 3, 800)",7.200
218,330,shufflenetv1/global_average_pooling2d_1/Mean,"(1, 800)",0.800
219,331,shufflenetv1/dense_1/MatMul;shufflenetv1/dense...,"(1, 3)",0.003
220,332,StatefulPartitionedCall:01,"(1, 3)",0.003


In [13]:
peak_memory_df = pd.read_pickle(models_peak_memory_df_path)
peak_memory_df

,operator,tensor_ids_in_memory,peak_memory_kb
1,shufflenetv1/re_lu_33/Relu;shufflenetv.../Bias...,"[0, 114]",82.944
2,shufflenetv1/max_pooling2d_1/MaxPool,"[114, 115]",69.120
3,shufflenetv1/average_pooling2d_3/AvgPool,"[115, 116]",17.280
4,shufflenetv1/re_lu_34/Relu;shufflenetv...1/dep...,"[115, 116, 117]",46.080
5,shufflenetv1/reshape_32/Shape,"[116, 117, 118]",32.272
...,...,...,...
216,shufflenetv1/re_lu_65/Relu;shufflenetv1/add_25...,"[328, 329, 316]",21.600
217,shufflenetv1/global_average_pooling2d_1/Mean,"[329, 330]",8.000
218,shufflenetv1/dense_1/MatMul;shufflenetv1/dense...,"[330, 331]",0.803
219,StatefulPartitionedCall:01,"[331, 332]",0.006


# Visualizations

In [18]:
# fig = px.bar(df, x="name", y= ["#i_1_activations","#i_2_activations", "#o_1_activations"], text_auto=".2s",
# width=1400, height=500,
#     title=f"Peak activations per layer - {model_name}")
# fig.add_hline(y=256000, line_width=3, line_dash="dash", line_color="orange", annotation_text="256 kB MCU constraint", 
#               annotation_position="bottom right")
# fig.update_layout(yaxis_range=[0,300000])
# #fig.update_layout(showlegend=True)
# fig.show()

In [24]:
tflite_fig = px.bar(peak_memory_df, x="operator", y= ["peak_memory_kb"], text_auto=".2s",
width=1400, height=600,
    title=f"Peak activations per layer - tflite {model_name}")
tflite_fig.add_hline(y=256, line_width=3, line_dash="dash", line_color="orange", annotation_text="256 kB MCU constraint", 
              annotation_position="bottom right")
peak_memory = peak_memory_df["peak_memory_kb"].max()
tflite_fig.add_hline(y=peak_memory, line_width=3, line_dash="dash", line_color="red", annotation_text=f"{peak_memory} kB peak activation", 
              annotation_position="top")
tflite_fig.update_layout(yaxis_range=[0,300])
#tflite_fig.update_layout(showlegend=True)
tflite_fig.show()

In [78]:
fig6 = px.bar(df, x="name", y= "MACS",
    title=f"MACs per layer - {model_name}")
#fig.add_hline(y=256000, line_width=3, line_dash="dash", line_color="orange", annotation_text="256 kB MCU constraint", 
#              annotation_position="bottom right")
#fig.update_layout(showlegend=True)
fig6.show()

In [74]:
fig2 = px.bar(df, x="name", y= "h_i_1", text_auto=".2s",
    title=f"Input height per layer - {model_name}")
#fig2.add_hline(y=256000, line_width=3, line_dash="dash", line_color="orange")
#fig2.update_yaxes(autorange="reversed")
#fig2.update_yaxes(rangemode="tozero")
fig2.show()

In [75]:
fig3 = px.bar(df, x="name", y= "c_i_1", text_auto=".2s",
    title=f"Channels per layer - {model_name}")
#fig2.add_hline(y=256000, line_width=3, line_dash="dash", line_color="orange")
#fig3.update_yaxes(autorange="reversed")
#fig3.update_yaxes(rangemode="tozero")
# fig3.update_yaxes(
#     range=(0, 1280),
#     constrain='domain'
# )
fig3.show()

In [76]:
fig4 = px.line(df, x="name", y= "total_params",
    title=f"Cumulative parameters per layer - {model_name}")
#fig.add_hline(y=256000, line_width=3, line_dash="dash", line_color="orange", annotation_text="256 kB MCU constraint", 
#              annotation_position="bottom right")
#fig.update_layout(showlegend=True)
fig4.show()

In [77]:
fig5 = px.line(df, x="name", y= "total_MACs",
    title=f"Cumulative MACs per layer - {model_name}")
#fig.add_hline(y=256000, line_width=3, line_dash="dash", line_color="orange", annotation_text="256 kB MCU constraint", 
#              annotation_position="bottom right")
#fig.update_layout(showlegend=True)
fig5.show()

In [ ]:
# # Generate run ids
# id = wandb.util.generate_id()

# PROJECT = model_name.split("_")[0]

# run = wandb.init(
#         # Set the project where this run will be logged
#         project=PROJECT, 
#         name = model_name,
#         id = id, 
#         resume="allow",
#         sync_tensorboard=True
#         )
# # Specify the configuration variables
# config = wandb.config

# #config.batch_size = BATCH_SIZE
# #config.dropout =DROPOUT
# #config.learn_rate = LR
# #config.momentum = MOMENTUM
# #config.decay = 1e-6
# #config.epochs = EPOCHS
# #config.classes = classes
# config.id = id
# config.architecture = model_name

# # Create a table
# table = wandb.Table(columns = ["plotly_figure"])

# # Create path for Plotly figure
# path_to_plotly_html = "./plotly_figure.html"

# # Write Plotly figure to HTML
# fig.write_html(path_to_plotly_html, auto_play = False) # Setting auto_play to False prevents animated Plotly charts from playing in the table automatically

# # Add Plotly figure as HTML file into Table
# table.add_data(wandb.Html(path_to_plotly_html))

# # Log Table
# run.log({"Chart_table": table})

# #wandb.log({"Peak activations chart": fig})

# wandb.finish()